In [1]:
import dotenv, os
import numpy as np
import pandas as pd
from supabase import create_client

# 시각화 라이브러리
import plotly.express as px
import plotly.graph_objects as go

# 차원 축소 라이브러리
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

dotenv.load_dotenv("../.env.local", override=True)

supabase_client = create_client(
    supabase_url=os.getenv("SUPABASE_URL"),
    supabase_key=os.getenv("SUPABASE_SECRET_KEY"),
)

In [2]:
# Supabase에서 모든 트랙 데이터 가져오기 (embedding 포함)
result = supabase_client.schema('public').table('tracks') \
    .select('id, title, tags, embeddings') \
    .execute()

print(f"총 {len(result.data)}개의 트랙 로드 완료")
print(f"샘플 데이터 키: {result.data[0].keys() if result.data else 'No data'}")

총 84개의 트랙 로드 완료
샘플 데이터 키: dict_keys(['id', 'title', 'tags', 'embeddings'])


In [7]:
# DataFrame으로 변환 및 embeddings 추출
tracks_df = pd.DataFrame(result.data)

# embedding이 None이 아닌 데이터만 필터링
tracks_df = tracks_df[tracks_df['embeddings'].notna()].reset_index(drop=True)

# 샘플 데이터 확인
print(f"embedding 타입: {type(tracks_df['embeddings'].iloc[0])}")
print(f"embedding 길이: {len(tracks_df['embeddings'].iloc[0]) if isinstance(tracks_df['embeddings'].iloc[0], (list, str)) else 'N/A'}")

# embeddings를 numpy 배열로 변환
# Supabase에서 가져온 데이터가 문자열인 경우 JSON 파싱 필요
import json

def parse_embedding(emb):
    """embedding을 list로 변환"""
    if isinstance(emb, str):
        return json.loads(emb)
    elif isinstance(emb, list):
        return emb
    return emb

embeddings_list = [parse_embedding(e) for e in tracks_df['embeddings']]
embeddings = np.array(embeddings_list)

print(f"유효한 트랙 수: {len(tracks_df)}")
print(f"Embeddings shape: {embeddings.shape}")


# tags 컬럼을 최대 15자까지만 잘라서 표시하는 컬럼 추가
tracks_df['tags_short'] = tracks_df['tags'].apply(lambda x: x[:20] + ('...' if len(x) > 15 else ''))

tracks_df[['id', 'title', 'tags_short']].head()

embedding 타입: <class 'str'>
embedding 길이: 6286
유효한 트랙 수: 84
Embeddings shape: (84, 512)


,id,title,tags_short
0,4,do re do re do,"fusion jazz, indie b..."
1,5,not really,"fusion jazz, catchy ..."
2,6,so,"fusion jazz, dreamy,..."
3,7,daily,"fusion jazz, happy, ..."
4,8,magnifier,"fusion jazz, dreamy,..."


In [8]:
# PCA로 3D 차원 축소
pca_3d = PCA(n_components=3, random_state=42)
embeddings_pca_3d = pca_3d.fit_transform(embeddings)

print(f"PCA 차원 축소 완료: {embeddings.shape} → {embeddings_pca_3d.shape}")
print(f"설명된 분산 비율: {pca_3d.explained_variance_ratio_}")
print(f"총 설명 분산: {sum(pca_3d.explained_variance_ratio_):.4f}")


PCA 차원 축소 완료: (84, 512) → (84, 3)
설명된 분산 비율: [0.28677535 0.17270928 0.12288557]
총 설명 분산: 0.5824


In [9]:
# PCA 3D 시각화 - Plotly 사용
fig_pca = px.scatter_3d(
    x=embeddings_pca_3d[:, 0],
    y=embeddings_pca_3d[:, 1],
    z=embeddings_pca_3d[:, 2],
    hover_name=tracks_df['title'],
    hover_data={'tags': tracks_df['tags_short']},
    title='🎵 Music Embeddings - PCA 3D Projection',
    labels={'x': 'PC1', 'y': 'PC2', 'z': 'PC3'},
)

fig_pca.update_traces(
    marker=dict(size=8, opacity=0.8),
    hovertemplate="<b>%{hovertext}</b><br>Tags: %{customdata[0]}<extra></extra>"
)

fig_pca.update_layout(
    width=900, 
    height=700,
    scene=dict(
        xaxis_title='PC1',
        yaxis_title='PC2',
        zaxis_title='PC3'
    )
)

fig_pca.show()


In [10]:
# t-SNE로 3D 차원 축소 (지역 구조 보존에 더 좋음)
# 데이터가 많으면 시간이 오래 걸릴 수 있음
print("t-SNE 3D 차원 축소 중... (시간이 걸릴 수 있습니다)")

tsne_3d = TSNE(
    n_components=3, 
    random_state=42, 
    perplexity=min(30, len(embeddings) - 1),  # perplexity는 데이터 수보다 작아야 함
    n_iter=1000,
    learning_rate='auto',
    init='pca'
)
embeddings_tsne_3d = tsne_3d.fit_transform(embeddings)

print(f"t-SNE 차원 축소 완료: {embeddings.shape} → {embeddings_tsne_3d.shape}")


t-SNE 3D 차원 축소 중... (시간이 걸릴 수 있습니다)


TypeError: TSNE.__init__() got an unexpected keyword argument 'n_iter'

In [ ]:
# t-SNE 3D 시각화 - Plotly 사용
fig_tsne = px.scatter_3d(
    x=embeddings_tsne_3d[:, 0],
    y=embeddings_tsne_3d[:, 1],
    z=embeddings_tsne_3d[:, 2],
    hover_name=tracks_df['title'],
    hover_data={'tags': tracks_df['tags']},
    title='🎵 Music Embeddings - t-SNE 3D Projection',
    labels={'x': 't-SNE 1', 'y': 't-SNE 2', 'z': 't-SNE 3'},
)

fig_tsne.update_traces(
    marker=dict(size=8, opacity=0.8, color='coral'),
    hovertemplate="<b>%{hovertext}</b><br>Tags: %{customdata[0]}<extra></extra>"
)

fig_tsne.update_layout(
    width=900, 
    height=700,
    scene=dict(
        xaxis_title='t-SNE 1',
        yaxis_title='t-SNE 2',
        zaxis_title='t-SNE 3'
    )
)

fig_tsne.show()


In [ ]:
# 태그에서 주요 장르 추출 (색상 분류용)
def extract_main_genre(tags: str) -> str:
    """태그 문자열에서 첫 번째 장르/스타일 추출"""
    if not tags or pd.isna(tags):
        return "Unknown"
    
    # 주요 장르 키워드
    genre_keywords = [
        'jazz', 'rock', 'pop', 'electronic', 'hip-hop', 'hiphop', 
        'lo-fi', 'lofi', 'ambient', 'indie', 'dance', 'house',
        'funk', 'soul', 'r&b', 'classical', 'folk', 'blues', 'phonk'
    ]
    
    tags_lower = tags.lower()
    for genre in genre_keywords:
        if genre in tags_lower:
            return genre.replace('-', ' ').title()
    
    return "Other"

# 장르 컬럼 추가
tracks_df['genre'] = tracks_df['tags'].apply(extract_main_genre)
print("장르 분포:")
print(tracks_df['genre'].value_counts())


In [ ]:
# 장르별 색상으로 t-SNE 3D 시각화
fig_tsne_genre = px.scatter_3d(
    x=embeddings_tsne_3d[:, 0],
    y=embeddings_tsne_3d[:, 1],
    z=embeddings_tsne_3d[:, 2],
    color=tracks_df['genre'],
    hover_name=tracks_df['title'],
    hover_data={'tags': tracks_df['tags']},
    title='🎵 Music Embeddings by Genre - t-SNE 3D Projection',
    labels={'x': 't-SNE 1', 'y': 't-SNE 2', 'z': 't-SNE 3', 'color': 'Genre'},
)

fig_tsne_genre.update_traces(
    marker=dict(size=8, opacity=0.85),
    hovertemplate="<b>%{hovertext}</b><br>Tags: %{customdata[0]}<extra></extra>"
)

fig_tsne_genre.update_layout(
    width=1000, 
    height=750,
    scene=dict(
        xaxis_title='t-SNE 1',
        yaxis_title='t-SNE 2',
        zaxis_title='t-SNE 3'
    ),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

fig_tsne_genre.show()
